In [4]:
%load_ext autoreload
%autoreload 2

import os
from filemanager import file_ext_search as fes
from dataclasses import dataclass, field
import numpy as np
import matplotlib.pyplot as plt
from scipy import signal

from pyproj import Proj, CRS, Transformer

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [5]:
from sbp_tools.sbp_track_processing import process_track, save_track_to_radex, save_track, read_segypos

crs_wgs84 = CRS.from_epsg(4326)
crs_utm35n = CRS.from_epsg(32635)
crs_utm34n = CRS.from_epsg(32634)

crs_world_mercator = CRS.from_epsg(3395)

transformer = Transformer.from_crs(crs_wgs84, crs_world_mercator, always_xy=True)

data_path = r'C:\Data\AI69\p012_Poly01_Ivan_conv'
# track
save_to = r'C:\Data\AI69\p012_Polyy01_Ivan.txt'

is_utm = False
is_arcsec = True
coord_scaler = 1
smooth = False
year_check = False
year = 2024

pos_files = fes.file_ext_search('.rawpos', data_path, recursive=True)
print(len(pos_files))
#pos_files = fes.file_ext_search('.txt', data_path, recursive=False)
#print(len(pos_files))

pos_files_LF = []
pos_files_LF = pos_files

#for file in pos_files:
#    if 'LF1' in file:
#        pos_files_LF.append(file)
    
    

segy_pos_objs = []
bad_data_dict = {}
fine_data_dict = {}

read_segypos(pos_files_LF, fine_data_dict, bad_data_dict, segy_pos_objs, year, utm_coords = is_utm, arcsecs=is_arcsec, coord_scale=coord_scaler)
process_track(segy_pos_objs, transformer, window_length=201, smooth=smooth, utm_coords=is_utm)
save_track(segy_pos_objs, save_to, smoothed=smooth)
save_track_to_radex(segy_pos_objs)

Searching *.rawpos files in directory:C:\Data\AI69\p012_Poly01_Ivan_conv
3
File ai69_id007_ABRPoly01_Pr is done 1 of 3
File ai69_id008_ABRPoly01_Pr is done 2 of 3
File ai69_id009_ABRPoly01_Pr is done 3 of 3


In [3]:
from shutil import move

move_to_bad_data = r'C:\Data\ABP56_FZ_UTM35N\sgy_ps3_SLF_FIXED_Bad'


for file in bad_data_dict.keys():
    segy_name = file[:-7]
    sgy_path = os.path.join(data_path, segy_name + '.sgy')
    pos_path = os.path.join(data_path,file + '.txt')
    
    move(sgy_path, move_to_bad_data)
    move(pos_path, move_to_bad_data)